In [239]:
import numpy

class matrix_factorization():
    
    def __init__(self,R,a,b,it,k):
        self.R=R
        self.user_n, self.item_n=R.shape
        self.alpha=a
        self.beta=b
        self.iterations=it
        self.K=k
        
    def learn(self):
        self.b_users=numpy.zeros(self.user_n)
        self.b_items=numpy.zeros(self.item_n)
        self.b=numpy.mean(self.R[numpy.where(self.R!=0)])
        
        self.P=numpy.random.normal(scale=1./self.K,size=(self.user_n,self.K))
        self.Q=numpy.random.normal(scale=1./self.K,size=(self.item_n,self.K))
        
        self.samples=[]
        for i in range (self.user_n):
            for j in range (self.item_n):
                if (self.R[i][j])>0:
                    self.samples.append((i,j,self.R[i][j]))

        for i in range (self.iterations):
            numpy.random.shuffle(self.samples)
            self.sgd()
        self.c=self.complete()
            
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            
            self.b_users[i]+=self.alpha*(e-self.beta*self.b_users[i])
            self.b_items[j]+=self.alpha*(e-self.beta*self.b_items[j])
            
            self.P[i,:]+=self.alpha*(e*self.Q[j,:]-self.beta*self.P[i,:])
            self.Q[j,:]+=self.alpha*(e*self.P[i,:]-self.beta*self.Q[j,:])
            
    def get_rating(self,i,j):
        return self.b_users[i]+self.b_items[j]+self.b+self.P[i,:].dot(self.Q[j,:].T)
    
    def complete(self):
        return self.P.dot(self.Q.T)+self.b+self.b_users[:,numpy.newaxis]+self.b_items[numpy.newaxis,:]
def Consensus(C):
    Cons=numpy.zeros((C[0].R.shape[0],C[0].R.shape[1],5))
    itc=numpy.nditer(Cons,flags=['multi_index'])
    for i in itc:
        y=(itc.multi_index[2]+1)
        for model in C:
            x=model.c[itc.multi_index[0]][itc.multi_index[1]]
            if(x==y):
                d=1
            else:
                d=1/abs(x-y)
            Cons[itc.multi_index[0]][itc.multi_index[1]][itc.multi_index[2]]+=d
    Cons=numpy.divide(Cons,len(C))
    for i in range (Cons.shape[0]):
        for j in range (Cons.shape[1]):
            norm=numpy.linalg.norm(Cons[i][j])
            Cons[i][j]=Cons[i][j]/norm
    return Cons

def kullback_leibler(C):
    K=numpy.zeros(C[0].R.shape)
    
    Cons=Consensus(C)
            
    itk=numpy.nditer(K,flags=['multi_index'],op_flags=['readwrite'])
    for x in itk:
        for model in C:
            p=[]
            for i in range(5):
                y=i+1
                if(model.c[itk.multi_index[0]][itk.multi_index[1]]==y):
                    d=1
                else:
                    d=1/abs(model.c[itk.multi_index[0]][itk.multi_index[1]]-y)
                d*=numpy.log(d/Cons[itk.multi_index[0]][itk.multi_index[1]][i])
                p.append(d)
            p=numpy.divide(p,numpy.linalg.norm(p))
            p=numpy.divide(p,len(C))
            s=p.sum()
        x+=s
    print(K)
    print(numpy.unravel_index(K.argmax(),K.shape))

In [240]:
M = numpy.array([
    [5, 2, 0, 1],
    [1, 2, 5, 1],
    [0, 3, 0, 1],
    [1, 0, 3, 4],
    [5, 1, 0, 2],
])
print(M)

mf5 = matrix_factorization(M, 0.1, 0.01, 20, 2)
mf1 = matrix_factorization(M, 0.1, 0.01, 20, 2)
mf4= matrix_factorization(M, 0.1, 0.01, 20, 2)
mf6 = matrix_factorization(M, 0.1, 0.01, 20, 2)
mf2 = matrix_factorization(M, 0.1, 0.01, 20, 2)
mf3 = matrix_factorization(M, 0.1, 0.01, 20, 2)

mf1.learn()
mf2.learn()
mf3.learn()
mf4.learn()
mf5.learn()
mf6.learn()

kullback_leibler([mf2,mf1,mf3,mf4,mf5,mf6])


[[5 2 0 1]
 [1 2 5 1]
 [0 3 0 1]
 [1 0 3 4]
 [5 1 0 2]]
[[0.20690935 0.25819452 0.2104429  0.29114491]
 [0.17850139 0.3075929  0.18363997 0.29021001]
 [0.17416147 0.33076441 0.20142411 0.31288788]
 [0.16782202 0.17334681 0.16675977 0.21438154]
 [0.23252698 0.31282383 0.21313912 0.2836545 ]]
(2, 1)
